In [6]:
import pandas as pd

In [7]:
data = pd.read_csv('result_mumbai.csv')

In [5]:
data

,area,latitude,longitude,bedrooms,bathrooms,balcony,ready_to_move,new_housing,parking,lift,flat_or_individual,furnished_status_Semi-Furnished,furnished_status_Unfurnished
0,629,19.032800,72.896357,2,2,0,0,1,0,0,1,0,1
1,974,19.032800,72.896357,3,2,0,0,1,0,0,1,0,1
2,968,19.085600,72.909277,3,3,0,0,1,0,0,1,0,1
3,629,19.155756,72.846862,2,2,2,1,1,2,2,1,0,1
4,1090,19.177555,72.849887,2,2,0,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6232,810,19.138320,72.810020,2,2,0,1,0,0,0,1,1,0
6233,1400,19.221920,72.854250,3,3,1,1,0,1,1,1,0,1
6234,750,19.144320,72.824111,2,2,0,1,0,0,0,1,0,1
6235,700,19.047201,72.872225,2,2,0,1,0,0,0,1,0,0


In [8]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

# Функция для обнаружения выбросов с использованием IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    non_outliers = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return non_outliers, outliers

# Функция для создания искусственных выбросов
def add_artificial_outliers(df, column, fraction=0.1, scale_factor=10):
    df_copy = df.copy()
    n_outliers = int(len(df_copy) * fraction)  # Количество выбросов (10% от подвыборки)
    outlier_indices = np.random.choice(df_copy.index, size=n_outliers, replace=False)
    df_copy.loc[outlier_indices, column] = df_copy.loc[outlier_indices, column] * scale_factor
    return df_copy

# Основная функция для создания подвыборок
def create_subsamples(df, target_column, min_size=200, max_size=600):
    # Перемешиваем датасет
    df = shuffle(df, random_state=42)
    
    # Определяем размер подвыборок
    sample_size = np.random.randint(min_size, max_size + 1)
    
    # 1. Подвыборка без выбросов
    non_outliers, _ = detect_outliers_iqr(df, target_column)
    subsample_no_outliers = non_outliers.sample(n=sample_size, random_state=42)
    
    # 2. Подвыборка с искусственными выбросами
    subsample_base = df.sample(n=sample_size, random_state=43)
    subsample_with_outliers = add_artificial_outliers(subsample_base, target_column)
    
    return subsample_no_outliers, subsample_with_outliers

In [9]:
np.random.seed(42)

In [20]:
subsample_no_outliers, subsample_with_outliers = create_subsamples(data, target_column='area')

In [21]:
subsample_with_outliers = subsample_with_outliers.reset_index(drop=True)

In [22]:
subsample_no_outliers = subsample_no_outliers.reset_index(drop=True)

In [23]:
y_with_out = subsample_no_outliers['price']
subsample_no_outliers = subsample_no_outliers.drop(columns=['price'])

In [30]:
y_with_out

0      19.00
1      15.00
2       9.80
3       2.50
4      12.00
       ...  
529    47.50
530     5.58
531    42.00
532     7.20
533    11.00
Name: price, Length: 534, dtype: float64

In [25]:
y_with = subsample_with_outliers['price']
subsample_with_outliers = subsample_with_outliers.drop(columns=['price'])

In [26]:
subsample_with_outliers.to_csv('with_outliers.csv')

In [29]:
y_with.to_csv('true_with_outliers.csv')

In [28]:
y_with_out.to_csv('true_no_outliers.csv')

In [10]:
subsample_no_outliers.value_counts()

area  latitude   longitude  bedrooms  bathrooms  balcony  ready_to_move  new_housing  parking  lift  flat_or_individual  furnished_status_Semi-Furnished  furnished_status_Unfurnished
1940  19.167035  72.881274  3         3          0        1              0            0        0     1                   0                                1                               1
525   19.071989  72.884082  2         2          0        1              0            0        0     1                   0                                1                               1
550   19.040233  72.860180  2         2          0        1              0            0        0     1                   0                                0                               1
      19.047201  72.872225  2         2          0        1              0            0        0     1                   1                                0                               1
600   19.015128  72.858064  2         2          1        1      

In [12]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

In [21]:
fig = px.histogram(subsample_with_outliers, x=subsample_with_outliers['area'], nbins=100)

In [22]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'bingroup': 'x',
              'hovertemplate': 'area=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#000001', 'pattern': {'shape': ''}},
              'name': '',
              'nbinsx': 100,
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': {'bdata': ('SAMAANMEAABMBAAAYgMAAH4EAADQaw' ... 'AA7gIAACADAACdCAAAogMAAJwCAAA='),
                    'dtype': 'i4'},
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'area'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}}}
})